In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
    # for filename in filenames:
    #     print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install ipython pandas numpy seaborn librosa soundfile scipy torch matplotlib nnAudio torchaudio tqdm Levenshtein transformers scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.9 MB/s eta 0:00:00a 0:00:01


In [3]:
import IPython.display as ipd
import pandas as pd
import numpy as np
import seaborn as sns
import librosa
import soundfile as sf
import re
import scipy
import torch
import matplotlib.pyplot as plt
import nnAudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import gc
import seaborn as sns

from glob import glob
from tqdm import tqdm
from nnAudio.features.mel import MelSpectrogram
from torchaudio.transforms import AmplitudeToDB
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC, 
    AutoProcessor, Wav2Vec2Model, HubertForCTC, 
    HubertModel, Data2VecAudioForCTC, Data2VecAudioModel,
    AutoModelForAudioXVector
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
%matplotlib inline

In [4]:
DEVICE = "cuda"

<a id='Homework'></a>
# Homework

Theory (8 points):
- Follow links.
- Try to fill/do **TODO** comments.
- Answer theory questions in the Google Form.

Practice (10 points):

#### Option 1 - Phoneme Recognition (PR)
Implement PR model using SSL features.

1. Take [TIMIT Dataset](https://www.kaggle.com/datasets/mfekadu/darpa-timit-acousticphonetic-continuous-speech).
2. Build model based on SSL features:
    - Try several SSL models — you can take models from lecture or/and explore [HF Model Hub](https://huggingface.co/models) or/and explore [Superb Leaderboard](https://superbbenchmark.org/leaderboard).
    - In lecture we have used `last_hidden_state`. Experiment with other layers, as an example check [Hubert/4 Experimental Details/C](https://arxiv.org/abs/2106.07447).
    - Experiment with different "Heads": Linear, MLP, Recurrent (GRU, LSTM), Conv, Attention. DO NOT use "too Deep" heads.
3. Use PER (Phonem Error Rate) for evaluation. Use pre-defined train/test split.

Example: https://www.kaggle.com/code/vladimirsydor/phoneme-recognition-with-wav2vec2.

#### Option 2 - Automatic Speech Recognition (ASR)
Implement ASR modle using SSLF features.

1. Take [OpenTTS UK dataset](https://huggingface.co/datasets/Yehor/opentts-uk)
2. Check **Option 1 description**
3. Evaluate model on [Common Voice 10 Test set](https://github.com/egorsmkv/cv10-uk-testset-clean). Use WER (Word Error Rate) and Character Error Rate (CER) for evaluation.

<a id='Optional_Additional_Task'></a>
# Optional Additional Task

**10 Points**

Voice Conversion based on:
- SSL features.
- KNN (K Nearest Neighbors).
- Vocoding from SSL features.

Resources:
- [Code](https://arxiv.org/abs/2305.18975).
- [Paper](https://github.com/bshall/knn-vc).

Apply the proposed approach using another SSL features (they have used WavLM):

- Try 2-3 another SSL features and different layers.
- Do Voice Conversion on VCTK.
- Train Vocoder on LIBRISPEECH or/and VCTK.
- Compare results with paper results. 

## Prepare Data

In [5]:
import pandas as pd
import os

train_data = pd.read_csv("../input/darpa-timit-acousticphonetic-continuous-speech/train_data.csv").dropna()
test_data = pd.read_csv("../input/darpa-timit-acousticphonetic-continuous-speech/test_data.csv").dropna()


In [6]:
train_data.head()

,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
0,1.0,TRAIN,DR4,MMDM0,SI681.WAV.wav,TRAIN/DR4/MMDM0/SI681.WAV.wav,TRAIN\\DR4\\MMDM0\\SI681.WAV.wav,True,True,False,False,False
1,2.0,TRAIN,DR4,MMDM0,SI1311.PHN,TRAIN/DR4/MMDM0/SI1311.PHN,TRAIN\\DR4\\MMDM0\\SI1311.PHN,False,False,False,True,False
2,3.0,TRAIN,DR4,MMDM0,SI1311.WRD,TRAIN/DR4/MMDM0/SI1311.WRD,TRAIN\\DR4\\MMDM0\\SI1311.WRD,False,False,True,False,False
3,4.0,TRAIN,DR4,MMDM0,SX321.PHN,TRAIN/DR4/MMDM0/SX321.PHN,TRAIN\\DR4\\MMDM0\\SX321.PHN,False,False,False,True,False
4,5.0,TRAIN,DR4,MMDM0,SX321.WRD,TRAIN/DR4/MMDM0/SX321.WRD,TRAIN\\DR4\\MMDM0\\SX321.WRD,False,False,True,False,False


In [7]:
data_path = '/kaggle/input/darpa-timit-acousticphonetic-continuous-speech/data/'

def combine_files(df):
    data = {}

    for idx, row in tqdm(df.iterrows()):
        path = row['path_from_data_dir']
        entry_id = path.split('.')[0]
    
        if entry_id not in data:
            data[entry_id] = {}
    
        if row['is_audio'] is True:
            data[entry_id]['audio_file'] = os.path.join(data_path, path)
        elif row['is_word_file'] is True:
            data[entry_id]['word_file'] = os.path.join(data_path, path)
        elif row['is_phonetic_file'] is True:
            data[entry_id]['phonetic_file'] = os.path.join(data_path, path)
    return data

In [8]:
combined_train_data = combine_files(train_data)
combined_test_data = combine_files(test_data)

8400it [00:00, 18383.52it/s]
8400it [00:00, 19210.05it/s]


In [9]:
train_df = pd.DataFrame.from_dict(combined_train_data, orient="index")
test_df = pd.DataFrame.from_dict(combined_test_data, orient="index")

val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42, shuffle=True)

In [10]:
train_df.head()

,audio_file,phonetic_file,word_file
TRAIN/DR4/MMDM0/SI681,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...
TRAIN/DR4/MMDM0/SI1311,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...
TRAIN/DR4/MMDM0/SX321,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...
TRAIN/DR4/MMDM0/SX51,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...
TRAIN/DR4/MMDM0/SX231,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...,/kaggle/input/darpa-timit-acousticphonetic-con...


In [11]:
train_df.shape

(1680, 3)

In [12]:
phon61_map39 = {
    'iy':'iy',  'ih':'ih',   'eh':'eh',  'ae':'ae',    'ix':'ih',  'ax':'ah',   'ah':'ah',  'uw':'uw',
    'ux':'uw',  'uh':'uh',   'ao':'aa',  'aa':'aa',    'ey':'ey',  'ay':'ay',   'oy':'oy',  'aw':'aw',
    'ow':'ow',  'l':'l',     'el':'l',  'r':'r',      'y':'y',    'w':'w',     'er':'er',  'axr':'er',
    'm':'m',    'em':'m',     'n':'n',    'nx':'n',     'en':'n',  'ng':'ng',   'eng':'ng', 'ch':'ch',
    'jh':'jh',  'dh':'dh',   'b':'b',    'd':'d',      'dx':'dx',  'g':'g',     'p':'p',    't':'t',
    'k':'k',    'z':'z',     'zh':'sh',  'v':'v',      'f':'f',    'th':'th',   's':'s',    'sh':'sh',
    'hh':'hh',  'hv':'hh',   'pcl':'h#', 'tcl':'h#', 'kcl':'h#', 'qcl':'h#','bcl':'h#','dcl':'h#',
    'gcl':'h#','h#':'h#',  '#h':'h#',  'pau':'h#', 'epi': 'h#','nx':'n',   'ax-h':'ah','q':'h#' 
}

In [13]:
def extract_file_data(df):
    data = []
    for i, row in df.iterrows():
        waveform, _ = torchaudio.load(row["audio_file"])
        phonetic_transcription_list = open(row["phonetic_file"]).readlines()
        word_transcription_list = open(row["word_file"]).readlines()

        phonetic_transcription = ""
        for phonetic_transcription_item in phonetic_transcription_list:
            phonetic_transcription += phon61_map39[phonetic_transcription_item.split(" ")[2].strip()] + " "

        word_transcription = ""
        for word_transcription_item in word_transcription_list:
            word_transcription += word_transcription_item.split(" ")[2].strip() + " "

        data.append([waveform, phonetic_transcription, word_transcription])
    return data

In [14]:
train_data = extract_file_data(train_df)

In [15]:
test_data = extract_file_data(test_df)

In [16]:
val_data = extract_file_data(val_df)

In [17]:
train_data[10]

[tensor([[ 0.0000e+00, -3.0518e-04, -1.8311e-04,  ...,  9.1553e-05,
           6.1035e-05,  6.1035e-05]]),
 'h# g ih dx ih h# k ae l ih h# k ow h# k ae h# t ah h# k iy h# p h# ',
 'get a calico cat to keep ']

In [18]:
phonetics = sorted(set([phone for x in train_data for phone in x[1].split()]))
ph2idx = {p: i for i, p in enumerate(phonetics)}
idx2ph = {i: p for i, p in ph2idx.items()}

In [20]:
def conv_ph2idx(data):
    for row in data:
        row[1] = [ph2idx[ph] for ph in row[1].split()]
    return data

In [21]:
train_data = conv_ph2idx(train_data)
test_data = conv_ph2idx(test_data)
val_data = conv_ph2idx(val_data)

In [22]:

class PhoneticDataset(Dataset):
    def __init__(self, data, num_phoneme_classes):
        """
        Initialize the dataset with multi-label phonetic encoding
        
        Args:
            data (list): List of [waveform, phonetic_transcription, word_transcription]
            num_phoneme_classes (int): Total number of unique phoneme classes
        """
        self.data = data
        self.num_phoneme_classes = num_phoneme_classes
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        Convert phonetic transcription to multi-label encoding
        
        Returns:
            tuple: (waveform, multi_label_phonetic_encoding, word_transcription)
        """
        waveform, phonetic_transcription, word_transcription = self.data[idx]
    
        
        return waveform, phonetic_transcription, word_transcription

In [23]:
train_ds = PhoneticDataset(train_data, len(phonetics))
test_ds = PhoneticDataset(test_data, len(phonetics))
val_ds = PhoneticDataset(val_data, len(phonetics))

In [24]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Unpack the batch
    waveforms, phonetic_labels, word_transcriptions = zip(*batch)
    
    # --- Audio Padding ---
    max_audio_length = max(waveform.shape[1] for waveform in waveforms)
    padded_audio = []
    attention_masks = []
    
    for waveform in waveforms:
        # Pad audio to max length
        pad_length = max_audio_length - waveform.shape[1]
        padded = torch.nn.functional.pad(waveform, (0, pad_length))
        
        # Create attention mask (1 for real audio, 0 for padding)
        mask = torch.ones_like(waveform, dtype=torch.long)
        mask = torch.nn.functional.pad(mask, (0, pad_length))
        
        padded_audio.append(padded)
        attention_masks.append(mask)
    
    # Stack audio and masks
    padded_audio = torch.stack(padded_audio)
    attention_masks = torch.stack(attention_masks)
    
    # --- Phonetic Label Padding ---
    label_lengths = torch.tensor([len(labels) for labels in phonetic_labels])
    padded_labels = pad_sequence(
        [torch.tensor(labels) for labels in phonetic_labels],
        batch_first=True,
        padding_value=-100
    )
    
    return padded_audio, attention_masks, padded_labels, label_lengths, word_transcriptions

In [25]:
batch_size = 4

train_dataloader = DataLoader(
    train_ds, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=3
)

test_dataloader = DataLoader(
    test_ds, 
    batch_size=batch_size, 
    collate_fn=collate_fn,
    num_workers=3
)

val_dataloader = DataLoader(
    val_ds, 
    batch_size=batch_size, 
    collate_fn=collate_fn,
    num_workers=3
)

In [26]:
for batch_idx, (audio_inputs, attention_masks, phonetic_labels,label_lengths, word_transcriptions) in enumerate(train_dataloader):
    audio_inputs = audio_inputs.to(DEVICE)
    attention_masks = attention_masks.to(DEVICE)
    print(phonetic_labels)
    phonetic_labels = phonetic_labels.to(DEVICE)
    break

tensor([[  15,    8,   17,   15,    7,   28,    2,   24,   15,   20,   11,    9,
           17,   38,   17,   29,   25,   30,   21,   15,    3,   15,   20,    1,
           29,   15,   31,   15, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100],
        [  15,    8,   18,   25,   35,   11,   36,   12,   15,    6,    0,   28,
           22,   11,   15,   20,   33,   15,    7,   29,   21,   17,   15,   27,
           26,   38,   23,   17,   23,   15,   31,   15,   10,   23,   18,   36,
            2,   23,   38,   15,   31,   18,   15],
        [  15,    8,   17,   15,   14,    2,   35,   11,   22,   17,   23,   15,
           29,    0,   15,    0,   32,   11,   17,   38,   12,   30,   17,   23,
           15,    2,   35,   16,   17,   29,   17,    9,   17,   29,   23,   30,
           17,   15,   15, -100, -100, -100, -100],
        [  15,    5,   28,    4,   15,   29,    2,   23,   30,    4,   23,   30,
           17,   22,   11,   38,  

## Test Wav2Vec

In [ ]:
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
# Refactored from https://chat.openai.com/share/e/52f6acc9-152d-4a99-b874-86998ad8fcc7
def load_and_process_audio(paths, sample_rate=None):
    # Load and resample audio files
    audios = []
    max_length = 0
    for path in tqdm(paths):
        audio, sr = librosa.load(path, sr=sample_rate)
        max_length = max(max_length, len(audio))
        audios.append(audio)

    # Pad audios to have the same length and stack them into a batch
    batched_audio = torch.zeros(len(audios), max_length)  # Assuming mono audio files
    padding_masks = torch.zeros(len(audios), max_length)
    for i, audio in enumerate(audios):
        length = len(audio)
        batched_audio[i, :length] = torch.from_numpy(audio)
        padding_masks[i, :length] = 1  # Mark non-padded areas as 1

    return batched_audio, padding_masks


In [ ]:
example_path = train_df.iloc[0]['audio_file']

In [ ]:
speaker_audio, speaker_pad_mask = load_and_process_audio([example_path])

In [ ]:
ipd.Audio(speaker_audio, rate = 16000)

In [ ]:
input_values = feature_extractor([
    el for el in speaker_audio.numpy()
], return_tensors="pt").input_values

In [ ]:
# Apply Model
with torch.no_grad():
    model_output = model(input_values.cuda())

In [ ]:
print(f"Input Shape [B, L]: {input_values.shape}")
print(f"Ouput Shape [B, N_L, C]: {model_output.logits.shape}")

In [ ]:
pred_ids = torch.argmax(model_output.logits[0], axis=-1)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
outputs.text

## Model

This model uses a frozen Wav2Vec 2.0 encoder (facebook/wav2vec2‑base‑960h) to turn raw audio into frame‑level feature vectors, then attaches a small, trainable head that maps those features to phoneme logits. 

You can choose a simple two‑layer MLP (halving the hidden size, ReLU + dropout, then projecting to your phoneme set) or an LSTM stack (for extra temporal context) before the final linear layer.

The head outputs log‑probabilities in the shape required by nn.CTCLoss(blank=0), so you don’t need frame‑level alignments. By freezing the Wav2Vec backbone you only learn a few hundred thousand head parameters, keeping training fast and stable.

In [27]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Model

class PhoneticRecognition(nn.Module):
    def __init__(self, num_phonemes, pretrained_model='facebook/wav2vec2-base-960h', use_lstm=False, lstm_hidden_size=512, num_layers=2):
        """
        Initialize Wav2Vec Phonetic Recognition Model with optional LSTM layer
        
        Args:
            num_phonemes (int): Number of unique phonemes in the target dataset
            pretrained_model (str): Pretrained wav2vec model to use as feature extractor
            use_lstm (bool): Whether to use LSTM or MLP for classification
            lstm_hidden_size (int): Hidden size for the LSTM (if use_lstm=True)
            num_layers (int): Number of LSTM layers (if use_lstm=True)
        """
        super(PhoneticRecognition, self).__init__()
        
        self.wav2vec = Wav2Vec2Model.from_pretrained(pretrained_model, output_hidden_states=True)
        
        for param in self.wav2vec.parameters():
            param.requires_grad = False
        
        feature_dim = self.wav2vec.config.hidden_size
        
        self.use_lstm = use_lstm
        
        if self.use_lstm:
            self.lstm = nn.LSTM(feature_dim, lstm_hidden_size, num_layers=num_layers, batch_first=False, dropout=0.5)
            self.classifier = nn.Linear(lstm_hidden_size, num_phonemes)
        else:
            self.classifier = nn.Sequential(
                nn.Linear(feature_dim, feature_dim // 2),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(feature_dim // 2, num_phonemes)
            )
        
    def forward(self, audio_input, attention_mask=None):
        """
        Forward pass for phonetic recognition
        
        Args:
            audio_input (torch.Tensor): Raw audio waveform
            attention_mask (torch.Tensor, optional): Mask for padded sequences
        
        Returns:
            torch.Tensor: Phoneme classification logits
        """
        audio_input = audio_input.squeeze()
        attention_mask = attention_mask.squeeze()

        outputs = self.wav2vec(audio_input, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states

        # tried some indices, 9 works better
        features = hidden_states[9]

        if self.use_lstm:
            lstm_out, _ = self.lstm(features)
            logits = self.classifier(lstm_out)  # (batch, seq_len, num_phonemes)
        else:
            logits = self.classifier(features)  # (batch, seq_len, num_phonemes)

        log_probs = torch.log_softmax(logits, dim=-1)
        return log_probs.permute(1, 0, 2)  # (seq_len, batch, num_phonemes)



In [28]:
import torch
import pytorch_lightning as pl


class PhoneticRecognitionModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model 
        self.learning_rate = learning_rate
        self.criterion = torch.nn.CTCLoss(blank=0)
        
    def forward(self, audio_inputs, attention_masks):
        return self.model(audio_inputs, attention_masks)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    
    def _compute_loss(self, batch):
        audio_inputs, attention_masks, phonetic_labels, phonetic_lengths, word_transcriptions = batch
        outputs = self(audio_inputs, attention_masks)
        
        batch_size = audio_inputs.shape[0]
        input_lengths = torch.full((batch_size,), outputs.shape[0])
        
        loss = self.criterion(outputs, phonetic_labels, input_lengths, phonetic_lengths)
        return loss
    
    def training_step(self, batch, batch_idx):
        loss = self._compute_loss(batch)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss = self._compute_loss(batch)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        loss = self._compute_loss(batch)
        self.log('test_loss', loss)
        return loss

In [29]:
pr_model_mlp = PhoneticRecognition(len(phonetics), use_lstm=False).to(DEVICE)

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

lightning_model_mlp = PhoneticRecognitionModel(pr_model_mlp)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='phonetic-recognition-{epoch:02d}-{val_loss:.4f}',
    save_top_k=1,
    mode='min'
)

logger = TensorBoardLogger("lightning_logs", name="phonetic_recognition")

trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback],
    logger=logger,
    accelerator='cuda' if torch.cuda.is_available() else 'cpu',
    devices=1
)


In [31]:
trainer.fit(lightning_model_mlp, train_dataloader, val_dataloader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [32]:
from typing import List

def edit_distance(a: List[int], b: List[int]) -> int:
    # classic Wagner–Fischer
    m, n = len(a), len(b)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m+1): dp[i][0] = i
    for j in range(n+1): dp[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,    # deletion
                dp[i][j-1] + 1,    # insertion
                dp[i-1][j-1] + cost  # substitution
            )
    return dp[m][n]

def calculate_per(hypothesis, reference):
    if not reference:
        return 0.0
    return edit_distance(hypothesis, reference) / len(reference)


In [33]:
def greedy_decode(log_probs, blank=0):
    preds = log_probs.argmax(dim=-1).transpose(0,1)
    decoded = []
    for seq in preds:
        tokens = []
        prev = None
        for t in seq.tolist():
            if t != prev and t != blank:
                tokens.append(t)
            prev = t
        decoded.append(tokens)
    return decoded

def evaluate_model(model, test_dataloader):
    model.eval()
    total_per = 0.0
    total_samples = 0

    with torch.no_grad():
        for audio, attn_mask, phonemes, phoneme_lens, _ in test_dataloader:
            audio = audio.to(DEVICE)
            attn_mask = attn_mask.to(DEVICE)

            log_probs = model(audio, attn_mask)
            hyps = greedy_decode(log_probs, blank=0)

            refs = []
            
            for i in range(phonemes.shape[0]):
                ref = phonemes[i].tolist()[:phoneme_lens[i]]
                refs.append(ref)

            # accumulate PER
            for hyp, ref in zip(hyps, refs):
                total_per += calculate_per(hyp, ref)
                total_samples += 1


    avg_per = total_per / total_samples if total_samples > 0 else 0.0
    print(f"PER on test dataset: {avg_per*100:.2f}%")
    return avg_per


In [34]:
# evaluate mlp head 
per = evaluate_model(pr_model_mlp.to('cuda'), test_dataloader)

PER on test dataset: 14.24%


In [35]:
pr_model_lstm = PhoneticRecognition(len(phonetics), use_lstm=True).to(DEVICE)
lightning_model_lstm = PhoneticRecognitionModel(pr_model_lstm)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback],
    logger=logger,
    accelerator='cuda' if torch.cuda.is_available() else 'cpu',
    devices=1
)

In [40]:
trainer.fit(lightning_model_lstm, train_dataloader, val_dataloader)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [41]:
# evaluate lstm head 
per = evaluate_model(pr_model_lstm.to(DEVICE), test_dataloader)

PER on test dataset: 14.51%


In our final evaluation, both the simple MLP head and the more complex LSTM head performed similarly, each achieving approximately a 14% error rate on the test set. Notably, both architectures started to overfit after around 10 epochs—even after increasing the dropout rate to 0.5 for both heads. Additionally, after testing the last hidden state as well as other intermediate layers, we found that using the 9th hidden state of Wav2Vec produced the best results for our phoneme recognition task.

Note: We also observed that the notebook interface does not display the loss values, so here are the recorded numbers for reference: for the MLP head, the validation loss was 0.243 and the epoch-level training loss was 0.148; for the LSTM head, the validation loss was 0.260 and the epoch-level training loss was 0.169.